## FIFTH EXPERIMENT

В пятом эксперименте мы используем уже улучешенные 2d keypoints, которые мы получили при помощи аппарата PixSfM. Для данных 2d keypoints нам известные tentative matching graphs (трэки). При помощи ray tracing мы находим соответствия между 2д и 3д точками. Теперь у нас есть трэки для соответсвующих 3д точек. При помощи различных методов по разреживанию графов мы избавляемся от outliers и находим среднюю точку для данного трека.


Пытаюсь построить кастомную геометричекскую верификацию мэтчей.

# Libraries

In [1]:
%%capture
!pip install ipython-sql 

In [33]:
%load_ext autoreload
%autoreload 2
%load_ext sql
import json

import sys

import open3d as o3d
import matplotlib.pyplot as plt
import torch
import h5py

from tqdm import tqdm

from collections import Counter
import pprint


import copy 
from collections import defaultdict
import pycolmap

from pathlib import Path
import numpy as np

sys.path.append('/workspace/sk3d/dev.sk_robot_rgbd_data/src')
sys.path.append("/workspace/pixel-perfect-sfm/")
sys.path.append("/workspace/pixel-perfect-sfm/Hierarchical-Localization")

from hloc import extract_features, match_features, match_dense, reconstruction, pairs_from_exhaustive, \
        visualization, pairs_from_retrieval
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap
from hloc.utils.read_write_model import  write_next_bytes, Point3D, Image, read_images_text, \
        read_points3D_binary, write_points3D_binary, write_images_binary, read_images_binary, \
        read_cameras_text

from pixsfm import ostream_redirect

from skrgbd.data.io import imgio
from skrgbd.calibration.camera_models import load_colmap_camera
from skrgbd.data.io.poses import load_poses

from skrgbd.data.processing.depth_utils.occluded_mesh_rendering import MeshRenderer


from utils import get_conf_upper_bound, draw_geometries

# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "all"
print(os.environ["CUDA_VISIBLE_DEVICES"])


import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)


pp = pprint.PrettyPrinter(depth=4)

all
1.11.0+cu113


# Setup

В **scene_name** необходимо задать имя объекта, над которым вы хотите провести эксперимент.

**delete_previous_output** - если True, то удаляет все предыдущие файлы в папке outputs. Использовать супер осторожно.

**has_cache** - если True, то у Вас уже существует файл с feature maps и он сохранен в папке cache_init. Это файл с feature maps Вы получаете только тогда, когда вы уже сделали featuremetric KA или BA для одного из ваших экспериментов.

**show_visualization** - если True, то показывает визуализацию результата эксперимента (3d pointcloud, задектированные keypoints (features) и final reprojections для какого-то изображения).

In [4]:
scene_name = 'dragon'
delete_previous_output = True

has_cache = True
show_visualization = False

**poses** - путь к файлу images.txt с известными позами камер (каждая вторая строка пустая)

**cameras_model** - путь к файлу cameras.txt, где хранятся GT параметры камеры

**images** - путь к папке с изображениями для реконструкции

**outputs** - путь к папке со всеми результатами

**cache_init** - путь к кэш-файлу, его мы получаем во время того, когда делаем KA или BA. В этот файле хранятся featuremaps после  dense feature extraction. В среднем на одну картинку размером 2368х1952 уходит 3 минуты. Этот файл вообще нельзя трогать, поэтому мы копируем его в папку outputs для своего эксперимента и продолжаем работу.

**cache_path** - тот же файл, что cache_init, с которым мы теперь будем работать во время эксперимента.

**sfm_pairs** - файл с названиями пар изображений на каждой строке

**features** - файл с features для каждой картинки, извлеченными при помощи feature_conf

**refined_keypoints** - файл с улучшенными 2d keypoints, файл был получен после процедуры KA (Ketpoint Adjustment)

**matches** - файл с matches для каждой пары картинок, извлеченными при помощи matcher_conf

In [24]:
root =  Path('/workspace')

#poses
poses = root / f'datasets/sk3d/dataset/{scene_name}/tis_right/rgb/images.txt'

#cam_model
camera_model = root / 'datasets/sk3d/dataset/calibration/tis_right/rgb/cameras.txt'

images = root / f'datasets/sk3d/dataset/{scene_name}/tis_right/rgb/undistorted/ambient@best'
outputs = root / f'pixel-perfect-sfm/outputs/{scene_name}/experiment_5_retrieval'

if delete_previous_output:
    !rm -rf $outputs 
    
outputs.mkdir(parents=True, exist_ok=True)  

if has_cache:
    cache_init = root / f'pixel-perfect-sfm/outputs/caches/{scene_name}/s2dnet_featuremaps_sparse.h5'
    !cp -r $cache_init $outputs
    cache_path = outputs / 's2dnet_featuremaps_sparse.h5'  

    
sfm_pairs = outputs / 'pairs-sfm.txt'      # top-k most covisible in SIFT model
loc_pairs = outputs / f'pairs-query-netvlad.txt'  # top-k retrieved by NetVLAD


features = outputs / 'features.h5'
refined_keypoints = outputs / 'refined_keypoints.h5'
matches = outputs / 'matches.h5'

# refined cameras

!cp -r /workspace/pixel-perfect-sfm/outputs/dragon/experiment_2/features.h5 $outputs
# !cp -r /workspace/pixel-perfect-sfm/outputs/dragon/experiment_2/refined/exp_not_refine_cam_params/refined_keypoints.h5 $outputs

!ls $outputs

features.h5  s2dnet_featuremaps_sparse.h5


In [34]:
exp5_dir = outputs / 'base'
exp5_dir.mkdir(parents=True, exist_ok=True)

!cp -r $poses $exp5_dir
!cp -r $camera_model $exp5_dir
!touch $exp5_dir/points3D.txt

!ls $exp5_dir

cameras.bin  cameras.txt  images.bin  images.txt  points3D.bin	points3D.txt


In [35]:
!colmap model_converter \
    --input_path $exp5_dir \
    --output_path $exp5_dir \
    --output_type BIN

!ls $exp5_dir

cameras.bin  cameras.txt  images.bin  images.txt  points3D.bin	points3D.txt


In [37]:
global_descriptors = extract_features.main(retrieval_conf, images, outputs)

pairs_from_retrieval.main(global_descriptors, loc_pairs, 5, db_model=exp5_dir)

features, loc_matches = match_dense.main(
    matcher_conf, 
    loc_pairs, 
    images, 
    outputs, 
    features=features, 
    max_kps=None,
    matches=matches)

[2022/12/05 12:39:03 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2022/12/05 12:39:03 hloc INFO] Found 100 images in root /workspace/datasets/sk3d/dataset/dragon/tis_right/rgb/undistorted/ambient@best.
[2022/12/05 12:39:03 hloc INFO] Skipping the extraction.
[2022/12/05 12:39:03 hloc INFO] Extracting image pairs from a retrieval database.
[2022/12/05 12:39:03 hloc INFO] Found 500 pairs.
[2022/12/05 12:39:03 hloc INFO] Extracting semi-dense features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


KeyError: 'preprocessing'

В папке **exp5_dir** будут сохранены все результаты данного эксперимента.

Смотрим при помощи **pycolmap** информацию о полученной на данной момент реконструкции.

In [ ]:
reconstruction = pycolmap.Reconstruction(exp5_dir)
print(reconstruction.summary())

Импортируем **hloc**. Она нам поможет создать БД, заполнить БД, сделать геометрическую верификацию (если понадобится).

In [ ]:
try:
    import hloc
except ImportError:
    print("Could not import hloc.")
    hloc = None

1) Создание БД

2) Импорт камер, картинок, улучшенных features, matches

Ниже клетка выполняется за 1 минуту.

In [ ]:
hloc_path = exp5_dir / 'hloc'
hloc_path.mkdir(parents=True, exist_ok=True)

database_path = hloc_path / 'database.db' 

print("DATABASE PATH -> ", database_path)

reference = pycolmap.Reconstruction(exp5_dir)   

images_txt_path = exp5_dir / 'images.txt'
images_dict = read_images_text(images_txt_path)
        
# Here I changed code and in database we have data about camera extrinsics    
image_ids = hloc.triangulation.create_db_from_model(reference, 
                                                    database_path, 
                                                    images_dict
                                                    )


#Importing features into database -> keypoints table 
hloc.triangulation.import_features(image_ids, 
                                   database_path, 
                                   refined_keypoints)


#Importing matches into database -> matches table

skip_geometric_verification = False
hloc.triangulation.import_matches(image_ids, 
                                  database_path, 
                                  sfm_pairs, 
                                  matches,
                                  min_match_score=None, 
                                  skip_geometric_verification=skip_geometric_verification)
#flag skip_geometric_verification == True - add info to two_view_geometry table, same matches are used.

In [ ]:
%reload_ext sql

%sql sqlite:////workspace/pixel-perfect-sfm/outputs/dragon/experiment_5/base/hloc/database.db

In [ ]:
%%sql

select count(*) from matches limit 1;

In [ ]:
%%sql

select sum(rows), count(*) from matches;

In [ ]:
skip_geometric_verification = False

estimate_two_view_geometries = True
verbose = True

if not skip_geometric_verification:
        if estimate_two_view_geometries:
            hloc.triangulation.estimation_and_geometric_verification(database_path, 
                                                  sfm_pairs, 
                                                  verbose)
        else:
            hloc.triangulation.geometric_verification(
                image_ids, 
                reference, 
                database_path, 
                features, 
                sfm_pairs, 
                matches)
            
# output ->  mean/med/min/max valid matches 98.71/99.13/65.79/100.00%.            

In [ ]:
%%sql

select sum(rows), count(*) from two_view_geometries;

In [ ]:
%%sql

select * from matches limit 1;

In [ ]:
%%sql

SELECT  count(*)
FROM    matches
JOIN
        two_view_geometries
ON      (matches.data = two_view_geometries.data);

In [ ]:
# %%sql

# SELECT m.pair_id
# FROM matches m
# WHERE EXISTS(SELECT g.pair_id 
# FROM two_view_geometries g
# WHERE g.data = m.data and g.pair_id = m.pair_id);

# Ray tracing

Здесь мы определяем наши 2D keypoints, которые считаем из нашей БД. 

(Повторяю, тут мы используем уже улучшенные 2D keypoints). 

2D keypoint в верхнем левом углу верхнего левого пикселя имеет координаты `(0, 0)`, и 2D keypoint в нижнем правом угля нижнего правого пикселя имеет координаты `(width, height)`.

In [ ]:
from typing import List, Tuple, Dict
from pathlib import Path
import numpy as np

from pixsfm.util.database import COLMAPDatabase, blob_to_array, pair_id_to_image_ids
from pixsfm.base import Map_NameKeypoints


def read_matches_from_db_modified(database_path: Path
                         ) -> Tuple[List[Tuple[str]], List[np.ndarray]]:
    db = COLMAPDatabase.connect(database_path)
    id2name = db.image_id_to_name()
    desc = {}
    for image_id, r, c, data in db.execute("SELECT * FROM descriptors"):
        d = blob_to_array(data, np.uint8, (-1, c))
        desc[image_id] = d / np.linalg.norm(d, axis=1, keepdims=True)
    # only compute scores if descriptors are in database
    compute_scores = (len(desc) > 0)
    scores = [] if compute_scores else None
    pairs = []
    matches = []
    for pair_id, data in db.execute("SELECT pair_id, data FROM two_view_geometries"):
        id1, id2 = pair_id_to_image_ids(pair_id)
        name1, name2 = id2name[id1], id2name[id2]
        if data is None:
            continue
        pairs.append((name1, name2))
        match = blob_to_array(data, np.uint32, (-1, 2))
        matches.append(match)
        if compute_scores:
            d1, d2 = desc[id1][match[:, 0]], desc[id2][match[:, 1]]
            scores.append(np.einsum('nd,nd->n', d1, d2))
    db.close()
    return pairs, matches, scores

In [ ]:
from pixsfm.util.colmap import read_keypoints_from_db, read_matches_from_db, write_keypoints_to_db

keypoints = read_keypoints_from_db(database_path)
#print(keypoints['0098.png'])

pairs, matches, scores = read_matches_from_db_modified(database_path) # the order in matches list is the same as in pairs

Загружаем данные: скана, модели камеры, позы камер.

In [ ]:
sk3d_root = '/workspace/datasets/sk3d'

dtype = torch.float
device = 'cpu'

rec = f'{sk3d_root}/dataset/{scene_name}/stl/reconstruction/cleaned.ply'
rec = o3d.io.read_triangle_mesh(rec)

occ = f'{sk3d_root}/dataset/{scene_name}/stl/occluded_space.ply'
occ = o3d.io.read_triangle_mesh(occ)

renderer = MeshRenderer(rec, occ); del occ

cam_model = f'{sk3d_root}/dataset/calibration/tis_right/rgb/cameras.txt'
cam_model = load_colmap_camera(cam_model).to(device, dtype)

poses = f'{sk3d_root}/dataset/{scene_name}/tis_right/rgb/images.txt'
world_to_cam = load_poses(poses, dtype).to(device, dtype)

Отрисуем 2D keypoints для первого изображения сцены.

In [ ]:
view_i = 0

key_pts = torch.tensor(keypoints['0000.png'], dtype=dtype)  # [[x1, y1], [x2, y2], ...]

# Visualize keypoints
img = f'{sk3d_root}/dataset/{scene_name}/tis_right/rgb/undistorted/ambient@best/{view_i:04}.png'
img = imgio.read.tis_right.rgb(img)

plt.figure(figsize=[15]*2)
plt.imshow(img)

print(key_pts)

plt.scatter(key_pts[:, 0], key_pts[:, 1], marker='x', c='yellow')

Трассировка лучей. Находим 2D keypoints с конечной hit depth.

In [ ]:
found_3d_points = {}

for image_id, image in reconstruction.images.items():
    
    key_pts = torch.tensor(keypoints[image.name], dtype=dtype)

    ray_dirs_in_cam_space = cam_model.unproject(key_pts.T.to(device, dtype))  # shape is 3, pts_n
    pts_n = ray_dirs_in_cam_space.shape[1]

    cam_to_world = world_to_cam[view_i].inverse()  # 4, 4
    cam_center_in_world_space = cam_to_world[:3, 3]  # 3
    ray_origins_in_world_space = cam_center_in_world_space.unsqueeze(0).expand(pts_n, -1)  # pts_n, 3
    cam_to_world_rot = cam_to_world[:3, :3]
    ray_dirs_in_world_space = ray_dirs_in_cam_space.T @ cam_to_world_rot.T  # pts_n, 3

    casted_rays = torch.cat([ray_origins_in_world_space, ray_dirs_in_world_space], 1)  # pts_n, 6
    renderer.occ_threshold = 1e-3
    
    hit_depth = renderer.render_rays(casted_rays, cull_back_faces=True)['ray_hit_depth']
    
    did_hit = hit_depth.isfinite()
    idxs_with_inf = (did_hit==False).nonzero().squeeze()
    
    pts_3d = (ray_dirs_in_world_space * hit_depth.unsqueeze(1) + ray_origins_in_world_space) 
    inf_array = np.asarray([np.inf, np.inf, np.inf])
    
    pts_3d[idxs_with_inf] = torch.from_numpy(inf_array).to(pts_3d) 
    found_3d_points[image.name] = list(pts_3d.numpy()) 

Save and check.



In [ ]:
o3d.io.write_triangle_mesh(str(outputs / 'scene.ply'), rec)

In [ ]:
# for idx, (name1, name2) in tqdm(enumerate(pairs)):
#     indices = [] 
#     collect_matches = []
#     for k, (idx1, idx2) in enumerate(list(matches[idx])):
#         if idx1 not in idx_to_remove_dict[name1] and idx2 not in idx_to_remove_dict[name2]:
#             collect_matches.append(np.asarray([idx1, idx2]))
#     matches[idx] = np.asarray(collect_matches)

In [ ]:
result_tracks = outputs / 'result' 
result_tracks.mkdir(parents=True, exist_ok=True)

# Tentative matching tracks

Создаем объект граф при помощи известных нам **pairs**, **matches**, **scores**. scores идут как None. 

In [ ]:
from pixsfm.keypoint_adjustment import build_matching_graph
from pixsfm import base, features, logger

graph = base.Graph()
scores = [None for _ in matches]

for (name1, name2), m, s in zip(pairs, matches, scores):
    graph.register_matches(name1, name2, m, s)   

print("Graph object ", graph)

Здесь мы создаем словарь, ключом которого является **track_label**, а значением список индексов его **nodes**.

В объекте граф есть **graph.nodes**. Это список всех nodes.

In [ ]:
# for i in [0, 1, 6464]:
#     print(graph.nodes[i]) 

In [ ]:
# Label graph
track_labels = base.compute_track_labels(graph)

track_labels_with_idxs = defaultdict(list)

for i, track_label in enumerate(track_labels):
    track_labels_with_idxs[track_label].append(i)

pp.pprint(track_labels_with_idxs)        

In [ ]:
len(graph.nodes)

In [ ]:
## working with just one track (example)

def find_3d_for_2d(image_id, 
                   feature_idx, 
                   points_3d):

    isBadPoint = False
    image_name = graph.image_id_to_name[image_id]
    
    if image_name not in points_3d:
        return None, True
        
    def is_inf_point(pts_3d):
        return np.allclose(np.isfinite(pts_3d), [False, False, False])
    
    pnt_3d = points_3d[image_name][feature_idx]
    if is_inf_point(pnt_3d): isBadPoint = True    
        
    return pnt_3d, isBadPoint


def collect_3d_points(graph, 
                     track_idxs,
                     found_3d_points,):
    
    """
        Collecting 3d points from tracks with same labels.
    """
    
    pts_in_tracks = defaultdict(list) # 3d-points
    
    for track_label in tqdm(track_idxs):
        idxs = track_idxs[track_label]
        
#         nodes = set()

        for i in idxs:
            feature_node = graph.nodes[i]      
            
#             node_idx = feature_node.node_idx
#             nodes.add(node_idx)
            
            pnt_3d, isBadPoint = find_3d_for_2d(feature_node.image_id, 
                                                feature_node.feature_idx, 
                                                found_3d_points)
           
            if pnt_3d is None and isBadPoint:
                continue
            
            elif not isBadPoint:  
                pts_in_tracks[track_label].append(pnt_3d)
                #print(track_label, feature_node, pnt_3d)
            else:
                pass
   
    return pts_in_tracks           

In [ ]:
pts_in_tracks = collect_3d_points(graph, 
                                  track_labels_with_idxs,
                                  found_3d_points,  
                                  )  

In [ ]:
pts_in_tracks.keys()

In [ ]:
for i in [1109, 1041]:
    print(track_labels_with_idxs[i], len(track_labels_with_idxs[i]))

In [ ]:
for i in [1109, 1041]:
    print(i, pts_in_tracks[i], len(pts_in_tracks[i]))
    print()

In [ ]:
from scipy.spatial import distance

plt.rcParams.update({'font.size': 12})

fig, axs = plt.subplots(20,5, figsize=(90, 350), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .1, wspace=.001)

axs = axs.ravel()

first100pairs = {k: pts_in_tracks[k] for k in list(pts_in_tracks)[:100]}

for i, (track_label, pts) in enumerate(first100pairs.items()):

    dist_matrix = distance.cdist(pts, pts, 'euclidean')
    
    upper_triangle_values = dist_matrix[np.triu_indices(len(pts), k = 1)]
    conf_upper_bound = get_conf_upper_bound(upper_triangle_values)
    
    bins = 10
    if len(upper_triangle_values) > 10: bins = 20

    axs[i].hist(upper_triangle_values, bins=bins)
    axs[i].axvline(conf_upper_bound, color='r', linestyle='--')
    axs[i].set_title(str(track_label))
    
fig.savefig(str(result_tracks / 'distribution_of_distances.pdf'))    
plt.close(fig)

In [ ]:
from scipy.spatial import distance

plt.rcParams.update({'font.size': 12})

dists = []

for i, (track_label, pts) in enumerate(pts_in_tracks.items()):

    dist_matrix = distance.cdist(pts, pts, 'euclidean')
    upper_triangle_values = dist_matrix[np.triu_indices(len(pts), k = 1)]
    
    if len(upper_triangle_values) != 0:
        dists.append(upper_triangle_values)
        
# conf_upper_bound = get_conf_upper_bound(upper_triangle_values)      
dists = [item for sublist in dists for item in sublist]
plt.hist(dists, bins=100)
plt.title("Distrbution of distances between points in all tracks")

fig.savefig(str(result_tracks / 'distribution_of_distances_in_all_tracks.pdf'))    

In [ ]:
!rm $result_tracks/pts_in_tracks_without_matches.json

_pts_in_tracks_without_matches = {}    

for k, val in pts_in_tracks.items():
    new_list = [el.tolist() for el in val]
    _pts_in_tracks_without_matches[k] = new_list 

with open( str(result_tracks / 'pts_in_tracks_without_matches.json'), 'w') as fp:
    json.dump(_pts_in_tracks_without_matches, fp)  

del _pts_in_tracks_without_matches    

### Checking track as a pointcloud in meshlab

In [ ]:
pointclouds = outputs / 'pointclouds'
pointclouds.mkdir(parents=True, exist_ok=True)

!rm -r $pointclouds

In [ ]:
pointclouds_tracks = outputs / 'pointclouds_tracks'

!rm -r $pointclouds_tracks
pointclouds_tracks.mkdir(parents=True, exist_ok=True)

In [ ]:
for idx, (track_label, pts) in enumerate(pts_in_tracks.items()):
    
    points = [p.tolist() for p in pts]

    pc = o3d.geometry.PointCloud()
    pc.points.extend(points)

    o3d.io.write_point_cloud(str(pointclouds_tracks / f'{track_label}_track.ply'), pc)

In [ ]:
for idx, (track_label, pts) in enumerate(pts_in_tracks.items()):
    
    _pts_in_tracks = copy.copy(pts_in_tracks)
    
    print(f"\nTRACK_LABEL_{track_label}")
    score_labels = base.compute_score_labels(graph, track_labels)

    scores_for_test = {}
    idxs = track_labels_with_idxs[track_label] 
    print("idxs",idxs,  "\n")

    for i in idxs: scores_for_test[i] = score_labels[i]

    print(scores_for_test)
    break
        
    sorted_by_score = dict(sorted(scores_for_test.items(), key=lambda item: item[1], reverse = True))
    print(sorted_by_score)
    
    for k, score in sorted_by_score.items():
        print(k, len(_pts_in_tracks[k]))
        
        if len(_pts_in_tracks[k]) != 0:

            points = [p.tolist() for p in _pts_in_tracks[k]]
            
            pc = o3d.geometry.PointCloud()
            pc.points.extend(points)
            
            track_folder = pointclouds / f'{track_label}'
            track_folder.mkdir(parents=True, exist_ok=True)
            
            o3d.io.write_point_cloud(str(track_folder / f'{track_label}_idx_{k}_score_{score}.ply'), pc)

In [ ]:
# for idx, (track_label, pts) in enumerate(pts_in_tracks.items()):
    
#     _pts_in_tracks = copy.copy(pts_in_tracks)
    
#     print(f"\nTRACK_LABEL_{track_label}")
#     score_labels = base.compute_score_labels(graph, track_labels)

#     scores_for_test = {}
#     idxs = track_labels_with_idxs[track_label] 
#     print("idxs",idxs,  "\n")

#     for i in idxs: scores_for_test[i] = score_labels[i]

#     print(scores_for_test)
#     break
        
#     sorted_by_score = dict(sorted(scores_for_test.items(), key=lambda item: item[1], reverse = True))
#     print(sorted_by_score)
    
#     for k, score in sorted_by_score.items():
#         print(k, len(_pts_in_tracks[k]))
        
#         if len(_pts_in_tracks[k]) != 0:

#             points = [p.tolist() for p in _pts_in_tracks[k]]
            
#             pc = o3d.geometry.PointCloud()
#             pc.points.extend(points)
            
#             track_folder = pointclouds / f'{k}'
#             track_folder.mkdir(parents=True, exist_ok=True)
            
#             o3d.io.write_point_cloud(str(track_folder / f'{track_label}_idx_{k}_score_{score}.ply'), pc)

### Plotting the distribution of points distances in each track and all tracks

In [ ]:
#plot number of points for each track

fig, ax1 = plt.subplots(1)

points_num_with = [len(value) for key, value in pts_in_tracks.items()]
# points_num_without = [len(value) for key, value in pts_in_tracks_without_matches.items()]

plt.rcParams["figure.figsize"] = (22,8)
ax1.hist(points_num_with, bins=100, alpha=0.7, label='with matches')
# ax2.hist(points_num_without, bins=100, alpha=0.3, label='without matches')

ax1.set_xlabel("# of points in track", fontsize = 14)
# ax2.set_xlabel("# of points in track", fontsize = 14)

ax1.legend()
# ax2.legend()

ax1.set_ylabel("Frequency", fontsize = 14)
# ax2.set_ylabel("Frequency", fontsize = 14)

fig.suptitle('Histogram of number of points in the tracks')

fig.savefig(str( result_tracks / 'number_of_points_in_the_tracks.pdf') )  

### Looking at tracks with highest scores

In [ ]:
# node within each track with highest score

root_labels = base.compute_root_labels(graph, track_labels, score_labels)
indices = [i for i, x in enumerate(root_labels) if x]

highest_score_tracks = {}

for i in indices:
    track_label = track_labels[i]
    highest_score_tracks[track_label] = [i]

In [ ]:
pts_in_highest_tracks = collect_3d_points(graph, 
                                  highest_score_tracks,
                                  found_3d_points,  
                                  addMatches=False)  

In [ ]:
pts_in_highest_tracks

In [ ]:
print(len(np.unique(list(pts_in_highest_tracks.values()), axis = 0)))

In [ ]:
pts_in_highest_tracks[466]
track_labels_with_idxs[466]   


def collect_feature_node_ids(track_labels_with_idxs):
    result_dict = defaultdict(list)
    for track_label, idxs in track_labels_with_idxs.items():
        for i in idxs:
            featurenode = graph.nodes[i]
            feature_num_matches = featurenode.num_matches
            
            feature_node_idx = featurenode.node_idx
            print(featurenode)
            break
            result_dict[track_label].append(featurenode_idx)
        break    

collect_feature_node_ids(track_labels_with_idxs)            

In [ ]:
pnts = [ el[0] for el in list(pts_in_highest_tracks.values())]

pc = o3d.geometry.PointCloud()
pc.points.extend(pnts)

print(len(pnts))

draw_geometries([pc])

## Method 0

In [ ]:
# Update points in track by mean value of given 3d points

upd_pts_method_1 = {}

for track_label, pts in pts_in_tracks.items():
    if len(pts) == 1:
        upd_pts_method_1[track_label] = pts
    if len(pts) >= 2:
        mean_pnt = np.mean(pts, axis = 0)
        upd_pts_method_1[track_label] = mean_pnt

## Method 1


In [ ]:
method1 = result_tracks / 'method1'
method1.mkdir(parents=True, exist_ok=True)

Ниже клетка выполняется за 45 минут.

In [ ]:
from scipy.spatial import distance
#  From distance fucntion distance matrix is returned. 
# For each i and j, the metric dist(u=XA[i], v=XB[j]) is computed and stored in the i,j entry.

from welzl import welzl

# compute the NxN inter-distances in O(N^2)
# sum each row of this matrix (= the distance of one point to the others) in O(N^2)
# sort the obtained "crowd" distance in O(N*log N)
# (the point with smallest distance is an approximation of the geometric median)
# remove the 5% largest in O(1)
# here we just consider largest crowd-distance as outliers,
# instead of taking the largest distance from the median.
# compute radius of obtained sphere in O(N)
# Of course, it also suffers from sub-optimality but should perform a bit better in case of far outlier.
# Overall cost is O(N^2).

upd_pts_method_1 = {}
radiuses_method_1 = []


for track_label, pts in tqdm(pts_in_tracks.items()):
    
    percentile_95 = round(0.95 * len(pts))
    dist_matrix = distance.cdist(pts, pts, 'euclidean')

    row_sum = np.sum(dist_matrix, axis=1)
    sorted_row_sum = np.sort(row_sum)    
    
    # here we just consider largest crowd-distance as outliers,
    # instead of taking the largest distance from the median.
    outliers_distances = sorted_row_sum[percentile_95:]
    
    idx_to_remove = []
    for dst in outliers_distances:
        result = np.where(row_sum == dst)[0][0]
        idx_to_remove.append(result)
    
    upd_pts =  []
    for i, p in enumerate(pts):
        if i not in idx_to_remove:
            upd_pts.append(p)
            
    nsphere = welzl(upd_pts)
    
    r = np.sqrt(nsphere.sqradius)
    cent = nsphere.center
    
    print("Track labes is ", track_label)
    print("    Center is at: ", cent)
    print("    Radius is: ", r, "\n")
    
    upd_pts_method_1[track_label] = cent
    radiuses_method_1.append(r)   
    
    del upd_pts
    
    
upd_pts_method_1 = {k: upd_pts_method_1[k].tolist() for k in list(upd_pts_method_1)}    

with open( str(method1 / 'method1_with_matches.json'), 'w') as fp:
    data = []
    data.append({"radiuses": radiuses_method_1})
    data.append({"3d_points": upd_pts_method_1})
    json.dump(data, fp)   

In [ ]:
def draw_distribution_of_radiuses(radiuses,
                                   method_num,
                                  path_to_method):
    plt.rcParams.update({'font.size': 12})

    x = radiuses
    y = range(len(radiuses))

    fig, (ax1, ax2, ax3) = plt.subplots(3)

    fig.set_figwidth(8)
    fig.set_figheight(15)

    ax1.set_title(f"Method {method_num}", fontsize=20)
    ax1.scatter(x, y)
    ax1.legend()

    ########################

    data = list(filter(lambda x: x != 0, radiuses)) # without 0 distance here!

    mean = np.mean(data)
    median = np.median(data)
    print("Mean ", mean)
    print("Median ", median)

    ax2.axvline(mean, color='r', linestyle='--')
    ax2.axvline(median, color='g', linestyle='-')

    ax2.hist(data, bins=50, density=True)

    ax2.set_xlabel("Radiuses of smallest bounding spheres")
    ax2.set_ylabel("Quantity")
    ax2.set_title("Distribution of found radiuses", fontsize=12)
    ax2.plot([], [], ' ', label=f"Mean: {mean}")
    ax2.plot([], [], ' ', label=f"Median: {median}")

    ax2.legend()


    #######################

    data_0 = radiuses

    mean = np.mean(data_0)
    median = np.median(data_0)
    print("Mean ", mean)
    print("Median ", median)

    ax3.axvline(mean, color='r', linestyle='--')
    ax3.axvline(median, color='g', linestyle='-')

    ax3.hist(data_0, bins=50, density=True)

    ax3.set_xlabel("Radiuses of smallest bounding spheres (with info about 0)")
    ax3.set_ylabel("Quantity")
    ax3.plot([], [], ' ', label=f"Mean: {mean}")
    ax3.plot([], [], ' ', label=f"Median: {median}")

    ax3.legend()

    fig.savefig(str( path_to_method / 'distribution_of_found_radiuses.pdf') )  

In [ ]:
draw_distribution_of_radiuses(radiuses_method_1, 1, method1)

## Method 2

In [ ]:
method2 = result_tracks / 'method2'
method2.mkdir(parents=True, exist_ok=True)

In [ ]:
import numpy as np
from scipy.stats import norm, nct

track_num = 1141

# a demo sample
x = [0.02130260546341465, 0.021878736423016517, 0.024428375041446026, 0.03167656248812453, 
     0.021607927798428992, 0.02218236952747051, 0.02938439390419522, 0.021895384443549402, 
     0.0208653394740238, 0.022615560017453837, 0.026069874576700663, 0.020490277066658413, 
     0.02276381086366452, 0.02741631464168221, 0.023065996816853965, 0.027920032580417257, 
     0.029783724542362115, 0.018672376360816232, 0.02411919130687289, 0.024032041697362883, 
     0.027668555080877243, 0.029038746866940128, 0.028905096038895173, 0.028455422389231457, 
     0.02777097119444502, 0.02645934696996008, 0.020406710342586114, 0.018363611315954646, 
     0.029764300669835594, 0.028150384772774876, 0.028483541898183005, 0.026539612194819227, 
     0.024770634678803787, 0.0256449404014951, 0.02701950540179652, 0.02800481701574395, 
     0.026255726719030262, 0.026822962223460232, 0.02795852919052212, 0.02867081143636126, 
     0.02802730758510055, 0.026939408443483283, 0.02756843844933439, 0.026711061791156974, 
     0.028354813127200527, 0.02717413194036283, 0.027048786517346646, 0.02093044948575734, 
     0.02828042330464175, 0.028437914098171705, 0.029003609002851094, 0.02791780539527151, 
     0.028315803118693243, 0.02760123113501837, 0.02773386028268046, 0.02857483708473208,
     0.029392926114190886, 0.027680620974533866, 0.020846311414102706, 0.021942669842570775, 
     0.02800255456735636, 0.028071564643575363, 0.019107524162219267, 0.02829224661377798, 
     0.028648979770239322, 0.028379039104801607, 0.02895200332650313, 0.027743450215637434, 
     0.02844947225303563, 0.028480928942857968, 0.027493861704528902, 0.03167656264724666]

n = len(x)

print("Sample size ", n)

conf_upper_bound = get_conf_upper_bound(x)
print("One-sided tolerance upper bound", conf_upper_bound)


plt.figure(figsize=(4, 5))
plt.axvline(conf_upper_bound, color='r', linestyle='--')
plt.hist(x, bins=len(x), density=True)


plt.title(f"Example for track number {track_num}")
plt.show()

Ниже клетка работает 1 час.

In [ ]:
# Update points via smallest bounding sphere (experiment 2) without percentile???

from scipy.spatial import distance
from welzl import welzl, dist

# I would iterate the following two steps until convergence:

# 1) Given a group of points, find the smallest sphere enclosing 100% of the points and work out its centre.
# 2) Given a centre, find the group of points containing 95% of the original number which is closest to the centre.
# Each step reduces (or at least does not increase) the radius of the sphere involved, so you can declare convergence when the radius stops decreasing.
# In fact, I would iterate from multiple random starts, each start produced by finding the smallest sphere that contains all of a small subset of points.
# I note that if you have 10 outliers, and you divide your set of points into 11 parts, at least one of those parts will not have any outliers.
# (This is very loosely based on https://en.wikipedia.org/wiki/Random_sample_consensus)

upd_pts_method_2 = {}
radiuses_method_2 = []

for track_label, pts in tqdm(pts_in_tracks.items()):
    
    nsphere = welzl(pts)
#     print("Track label is ", track_label)
#     print("    Center is at: ", nsphere.center)
#     print("    Radius is: ", np.sqrt(nsphere.sqradius),)
    
    center = nsphere.center
    dists_to_center = [dist(p, center) for p in pts]
    sort_dists = np.sort(dists_to_center)

    upper_bound = get_conf_upper_bound(dists_to_center)
    out_indx = [i for i,v in enumerate(dists_to_center) if v > upper_bound]
    
    upd_pts =  []
    
    if len(out_indx) != 0:
        for i, p in enumerate(pts):
            if i not in out_indx:
                upd_pts.append(p)
    else:
        upd_pts = pts
             
            
    nsphere = welzl(upd_pts)
    #print("For points: ", upd_pts)
    print("Track labes is ", track_label)
    print("    Center is at: ", nsphere.center)
    print("    Radius is: ", np.sqrt(nsphere.sqradius), "\n")
    
    upd_pts_method_2[track_label] = nsphere.center
    radiuses_method_2.append(np.sqrt(nsphere.sqradius))   
    
    
upd_pts_method_2 = {k: upd_pts_method_2[k].tolist() for k in list(upd_pts_method_2)}    

with open( str(method2 / 'method2_with_matches.json'), 'w') as fp:
    data = []
    data.append({"radiuses": radiuses_method_2})
    data.append({"3d_points": upd_pts_method_2})
    json.dump(data, fp)       

In [ ]:
draw_distribution_of_radiuses(radiuses_method_2,
                                   2,
                                  method2)

## Method 3

In [ ]:
method3 = result_tracks / 'method3'
method3.mkdir(parents=True, exist_ok=True)

In [ ]:
# Stat with sum of pair-wise distances

from scipy.spatial import distance
from welzl import welzl

#  From distance fucntion distance matrix is returned. 
# For each i and j, the metric dist(u=XA[i], v=XB[j]) is computed and stored in the i,j entry.

# compute the NxN inter-distances in O(N^2)
# sum each row of this matrix (= the distance of one point to the others) in O(N^2)
# sort the obtained "crowd" distance in O(N*log N)
# (the point with smallest distance is an approximation of the geometric median)
# remove the 5% largest in O(1) with statistics here
# compute radius of obtained sphere in O(N)
# Of course, it also suffers from sub-optimality but should perform a bit better in case of far outlier.
# Overall cost is O(N^2).

upd_pts_method_3 = {}
radiuses_method_3 = []
init_radiuses = []

for track_label, pts in tqdm(pts_in_tracks.items()):
    
    nsphere = welzl(upd_pts)
    init_radiuses.append(nsphere.center)
    
    dist_matrix = distance.cdist(pts, pts, 'euclidean')

    row_sum = np.sum(dist_matrix, axis=1)
    #print(row_sum)

    sorted_row_sum = np.sort(row_sum)    
#     print("Distances: ", sorted_row_sum)
    
    upper_bound = get_conf_upper_bound(row_sum)
    out_indx = [i for i,v in enumerate(row_sum) if v > upper_bound]
    
    upd_pts = []
    if len(out_indx) != 0:
        for i, p in enumerate(pts):
            if i not in out_indx:
                upd_pts.append(p)
    else:
        upd_pts = pts
    
    nsphere = welzl(upd_pts)
    
    print("Track labes is ", track_label)
    print("    Center is at: ", nsphere.center)
    print("    Radius is: ", np.sqrt(nsphere.sqradius), "\n")
    
    upd_pts_method_3[track_label] = nsphere.center
    radiuses_method_3.append(np.sqrt(nsphere.sqradius))    
    
upd_pts_method_3 = {k: upd_pts_method_3[k].tolist() for k in list(upd_pts_method_3)}    

with open( str(method3 / 'method3_with_matches.json'), 'w') as fp:
    data = []
    data.append({"radiuses": radiuses_method_3})
    data.append({"3d_points": upd_pts_method_3})
    json.dump(data, fp)    
    

In [ ]:
draw_distribution_of_radiuses(radiuses_method_3, 3, method3)

In [ ]:
# https://www.geeksforgeeks.org/how-to-plot-normal-distribution-over-histogram-in-python/

data1 = radiuses_method_1
data2 = radiuses_method_2
data3 = radiuses_method_3

plt.hist(data1, bins=20, alpha=0.7, label='method1')
plt.hist(data2, bins=20,  alpha=0.3, label='method2')
plt.hist(data3, bins=20,  alpha=0.8, label='method3')

plt.xlabel("Radiuses of smallest bounding spheres")
plt.ylabel("Quantity")
plt.legend()
plt.title(f"Distribution of found radiuses", fontsize=18)

plt.savefig(str(result_tracks / f'distribution_of_radiuses_in_3_methods.pdf'))  

## Method  4

In [ ]:
method4 = result_tracks / 'method4'
method4.mkdir(parents=True, exist_ok=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN


data = np.asarray([el.tolist() for el in pts_in_tracks[1041]])

fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(data[:,0], data[:,1], data[:,2], s=300)
ax.view_init(azim=200)
plt.show()



# eps - The maximum distance between two samples for one to be considered as in the neighborhood of the other.
# This is not a maximum bound on the distances of points within a cluster. 
# This is the most important DBSCAN parameter to choose appropriately for your data set and distance function.

model = DBSCAN(eps=0.01, min_samples=2)
model.fit_predict(data)
pred = model.fit_predict(data)

fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(data[:,0], data[:,1], data[:,2], c=model.labels_, s=300)
ax.view_init(azim=200)
plt.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)

## Method 5 

In [ ]:
method5 = result_tracks / 'method5'
method5.mkdir(parents=True, exist_ok=True)

In [ ]:
upd_pts_method_5 = {}
radiuses_method_5 = []
init_radiuses = []

for track_label, pts in tqdm(pts_in_tracks.items()):
    
    print("Track label is ", track_label, " len(points): ", len(pts))
    
    if len(pts) in list(range(3)):
        
        nsphere = welzl(upd_pts)
        sphere_center = nsphere.center
        sphere_radius = np.sqrt(nsphere.sqradius)
        
        print("    Center is at: ", sphere_center)
        print("    Radius is: ", sphere_radius, "\n")
        
    else:
        
        _pts = copy.deepcopy(pts)
        
        nsphere = welzl(_pts)
        prev_radius = np.sqrt(nsphere.sqradius)
        new_radius = 0
        new_center = 0
        
        MAX_ITERS = 5
        
        while MAX_ITERS > 0 and abs(prev_radius - new_radius) >= 0.001:

            radiuses = []
            new_set_points = []

            for i, point in enumerate(_pts):
                new_set_points = [_pts[k] for k in range(len(_pts)) if k != i]
                new_nsphere = welzl(new_set_points)
                radiuses.append(new_nsphere.sqradius)

            index_max = np.argmax(radiuses)
            _pts.pop(index_max)  
            
            
            nsphere = welzl(_pts)
            new_center = nsphere.center
            new_radius = np.sqrt(nsphere.sqradius)
            
            if new_radius < prev_radius:
                prev_radius = new_radius
            
            
            MAX_ITERS -= 1
        
        nsphere = welzl(_pts)
        sphere_center = nsphere.center
        sphere_radius = np.sqrt(nsphere.sqradius)


    print("    Center is at: ", nsphere.center)
    print("    Radius is: ", np.sqrt(nsphere.sqradius), "\n")
    
    upd_pts_method_5[track_label] = sphere_center 
    radiuses_method_5.append(sphere_radius)      
    
    
        
    
# upd_pts_method_5 = {k: upd_pts_method_5[k].tolist() for k in list(upd_pts_method_5)}    

# with open( str(method5 / 'method5.json'), 'w') as fp:
#     data = []
#     data.append({"radiuses": radiuses_method_5})
#     data.append({"3d_points": upd_pts_method_5})
#     json.dump(data, fp)    
    

In [ ]:
draw_distribution_of_radiuses(radiuses_method_5, 5, method5)

In [ ]:
# https://www.geeksforgeeks.org/how-to-plot-normal-distribution-over-histogram-in-python/

data1 = radiuses_method_1
data2 = radiuses_method_2
data3 = radiuses_method_3
data5 = radiuses_method_5

plt.hist(data1, bins=50, alpha=0.7, label='method1')
plt.hist(data2, bins=50,  alpha=0.3, label='method2')
plt.hist(data3, bins=50,  alpha=0.9, label='method3')
plt.hist(data5, bins=50,  alpha=0.5, label='method5')

plt.xlabel("Radiuses of smallest bounding spheres")
plt.ylabel("Quantity")
plt.legend()
plt.title(f"Distribution of found radiuses", fontsize=18)

plt.savefig(str(result_tracks / f'distribution_of_radiuses_in_4_methods.pdf'))  

I compute track labels to see how many graph nodes, grap edges, tracks I have. Each track has its own index, here we have 1310 tracks,

In [ ]:
# Label graph
track_labels = base.compute_track_labels(graph)

track_count = Counter(track_labels)
print(track_count, len(track_count))

print("Track labels length: ", len(track_labels), "\nMax track_label: ", max(track_labels))

max_per_problem = max(track_count.values())
print("max_per_problem ", max_per_problem, sorted(track_count.keys()))
print(sum(track_count.values()))

In [ ]:
# score to connected features within same track
score_labels = base.compute_score_labels(graph, track_labels)
print(score_labels, len(score_labels))

In [ ]:
score_labels

In [ ]:
graph = ()
n_set = []

track_labels - tarck number
for i in track_labels:
    graph.nodes[i] -> track_label_i
    gather same track_labels_i -> n_set[i]
    
for cluster in sets:    
    build graph from gathered set
    run through teh graph to find the biggest edge

1) окружность
2) посмотреть про центр масс для класстера
3) статистика по сумму длин ребер (распредение расстояний)

4) https://medium.com/dive-into-ml-ai/faster-implementation-of-mahalanobis-distance-using-tensorflow-42f7aa586bac
    scipy.spatial.distance.mahalanobis
    
5) http://www.open3d.org/docs/release/tutorial/geometry/pointcloud_outlier_removal.html#Radius-outlier-removal
    
6) https://www.datatechnotes.com/2020/04/anomaly-detection-with-elliptical-envelope-in-python.html
    elliptical envilope